# Clean CA Data

In [1]:
import pandas as pd
import numpy as np
from siuba import *

In [2]:
from calitp import to_snakecase

In [3]:
pd.set_option('display.max_columns', None)
#pd.options.display.float_format = '{:,.2f}'.format
#pd.set_option('display.max_colwidth', None)

In [4]:
#!pip install lxml

In [5]:
# help from: https://towardsdatascience.com/how-to-get-tables-from-websites-into-pandas-dataframes-e82bd8a0ac59
aw = (pd.read_html('https://cleancalifornia.dot.ca.gov/local-grants')[0])
aw = aw.dropna(how = 'all')
aw = to_snakecase(aw)

In [6]:
aw['app_fy'] = 2021

In [7]:
aw.head()

,agency_name,project_title,grant_amount,total_project_cost,project_description,app_fy
1,Alameda County District Attorney's Office,Alameda County District Attorney’s Clean Stree...,"$2,369,091","$3,010,069",The Alameda County District Attorney’s (DA) of...,2021
2,City of Anaheim,"North La Palma Green Promenade, Paseo and Park...","$2,639,009","$3,518,679",Clean California Funds will allow the City of ...,2021
3,City of Bakersfield,Garces Memorial Circle Enhancement Project,"$1,491,250","$1,491,250",The Garces Memorial Circle is a roundabout tha...,2021
4,City of Bell Gardens,BG Keep Parks Clean Beautification Project,"$510,067","$510,067",The project will improve Hannon Park and Julia...,2021
5,City of Berkeley,Martin Luther King Jr. Civic Center Park - Tur...,"$591,666","$946,666",The Project will convert the existing defunct ...,2021


## Application Data

In [7]:
## read in Clean CA applications 
# NOTE: do not need this anymore

In [8]:
# apps = pd.read_excel('cc-lgp-app-log-order-recd-v3.xlsx')
# apps = apps.dropna(how='all')
# apps = apps.rename(columns = {'Implementing Agency Name': 'Agency Name',
#                              'Project Title ':'Project Title'})
# apps = to_snakecase(apps)

In [9]:
#apps

In [10]:
#apps['app_year'] = 2021

In [11]:
#apps

### joins 

In [12]:
# test = aw.merge(
#     apps,
#     how="outer",
#     on =["agency_name", "project_title"],
#     indicator=True,
# )

In [13]:
# test._merge.value_counts()

#### Reminder:
* right = application list 2021
* left = awarded list 
* both = awarded in 2021

should be 105 awarded projects in the both category

In [14]:
# (test >> filter(_._merge  != ("both"))>>arrange(_.agency_name))

In [56]:
### merging again on project title

In [15]:
# test2 = aw.merge(
#     apps,
#     how="outer",
#     on = "project_title",
#     indicator=True,
# )

In [16]:
# test2._merge.value_counts()

In [17]:
# (test >> filter(_._merge == ("both")) >> arrange(_.agency_name))

## Preliminary Analysis

In [15]:
print(f"There are {len(aw>>count(_.agency_name))} unique agencies that received Clean CA funds")

There are 87 unique agencies that received Clean CA funds


In [12]:
aw>>count(_.agency_name)>>arrange(-_.n)>>filter(_.n>1)

,agency_name,n
48,City of Sacramento,5
17,City of Glendale,3
29,City of Los Angeles,3
68,County of Kern,3
73,County of Tuolumne,3
36,City of Oakland,2
40,City of Pico Rivera,2
41,City of Pittsburg,2
45,City of Richmond,2
47,City of Rio Dell,2


In [25]:
#aw.grant_amount.astype('Int64')

In [30]:
aw['total_project_cost'] = aw.total_project_cost.replace('[\$,]', '', regex=True).astype(int)

In [32]:
aw['grant_amount'] = aw.grant_amount.replace('[\$,]', '', regex=True).astype(int)

In [33]:
aw['remaining_cost'] = (aw['total_project_cost'] - aw['grant_amount'])

In [45]:
aw

,agency_name,project_title,grant_amount,total_project_cost,project_description,app_fy,remaining_cost
1,Alameda County District Attorney's Office,Alameda County District Attorney’s Clean Stree...,2369091,3010069,The Alameda County District Attorney’s (DA) of...,2021,640978
2,City of Anaheim,"North La Palma Green Promenade, Paseo and Park...",2639009,3518679,Clean California Funds will allow the City of ...,2021,879670
3,City of Bakersfield,Garces Memorial Circle Enhancement Project,1491250,1491250,The Garces Memorial Circle is a roundabout tha...,2021,0
4,City of Bell Gardens,BG Keep Parks Clean Beautification Project,510067,510067,The project will improve Hannon Park and Julia...,2021,0
5,City of Berkeley,Martin Luther King Jr. Civic Center Park - Tur...,591666,946666,The Project will convert the existing defunct ...,2021,355000
...,...,...,...,...,...,...,...
101,Riverside University Health System-Public Health,"Clean, Beautiful and Resilient Oasis",2150942,2150942,This proposal will address infrastructure and ...,2021,0
102,Santa Ynez Chumash Tribe,Outdoor Space,303800,303800,The Santa Ynez Band of Chumash Indians will re...,2021,0
103,San Francisco Municipal Transportation Agency,South of Market (SoMa) Transit Gardens Connect...,1824328,3649395,Funds from Clean California will support the S...,2021,1825067
104,Southgate Recreation & Park District,South Sacramento Parks & Pathways Beautificati...,5000000,5001512,The Southgate Recreation & Park District (Sout...,2021,1512


In [41]:
print(f"{len(aw>>filter(_.remaining_cost==0)>>count(_.agency_name))} agencies project costs are covered by the grant amount.")

50 agencies project costs are covered by the grant amount.


#### exporting to parquet

In [47]:
#aw.to_parquet('cleanca.parquet')